## 2018-2022 Nationwide election results, by precinct

This pulls nationwide precinct level results from 2018, 2020, and 2022 from the
MIT Election Science Lab, from several different files:
- 2018 State: https://doi.org/10.7910/DVN/ZFXEJU
- 2018 US Senate: https://doi.org/10.7910/DVN/DGNAFS
- 2018 US House: https://doi.org/10.7910/DVN/IVIXLK
- 2020 State: https://doi.org/10.7910/DVN/OKL2K1
- 2020 US House: https://doi.org/10.7910/DVN/VLGF2M
- 2020 US Senate: https://doi.org/10.7910/DVN/ER9XTV
- 2020 President: https://doi.org/10.7910/DVN/JXPREB
- 2022 All Levels: https://github.com/MEDSL/2022-elections-official/tree/ad541ea94cb6697c0375a8cffe8e91b60372f50b/individual_states
   not published on dataverse yet, download from github.

I tried to use [State Precinct-Level Returns 2016](https://doi.org/10.7910/DVN/GSZG1O),
but this is ain a different format from the later ones, and can't be parsed by duckdb:
https://github.com/duckdb/duckdb/issues/10929

I tried for a bit using the [programmatic API](https://guides.dataverse.org/en/5.5/api/dataaccess.html#download-by-dataset-api)
to download, but it wasn't working so just gave up.

It also does some additional cleanup of that raw data, eg
- normalize date formats and fips codes.
- normalize "APPROVE" and "FOR" to a simple "YES"
- fill in NULLs when it seems correct to do so

I don't drop any rows or otherwise lose information.
    

In [1]:
from pathlib import Path

import ibis
from ibis import _

ibis.options.interactive = True

In [2]:
def parse_date(s):
    # some dates are in the format "mm/dd/yyyy" instead of "yyyy-mm-dd"
    # https://github.com/MEDSL/2022-elections-official/issues/20
    s = s.re_replace(r"(\d\d?)/(\d\d?)/(20)?(\d\d)", r"20\4-\1-\2")
    return s.cast("date")


assert parse_date(ibis.literal("01/02/20")).cast(str).execute() == "2020-01-02"
assert parse_date(ibis.literal("11/8/22")).cast(str).execute() == "2022-11-08"
assert parse_date(ibis.literal("01/02/2020")).cast(str).execute() == "2020-01-02"
assert parse_date(ibis.literal("2020-01-02")).cast(str).execute() == "2020-01-02"


def parse_fips5(s):
    s = s.nullif("NAN")
    s = s.nullif("NA")
    s = s.cast(int).cast(str)
    # ensure all FIPS codes are 5 digits
    s = s.re_replace(r"(\d{4})", r"0\1")
    return s

In [3]:
schema = {
    "year": "int16",
    "date": "date",
    "state_po": "string",
    # All of the following are duplicate info from state_po that just take up space.
    # You can look these up pretty easily if you need them.
    # "state": "string",
    # "state_fips": "string",
    # "state_cen": "string",
    # "state_ic": "string",
    "county_name": "string",
    "county_fips": "string",
    "jurisdiction_name": "string",
    "jurisdiction_fips": "string",
    "district": "string",
    "office": "string",
    "magnitude": "int",  # the number seats available for this office
    "special": "boolean",
    "stage": "string",
    "precinct": "string",
    "writein": "boolean",
    "candidate": "string",
    "party_detailed": "string",
    # "party_simplified": "string",  # this can be derived from party_detailed
    "mode": "string",  # how the votes were cast eg "EARLY", "ABSENTEE", etc
    # can't cast to int because some rows contain "*":
    # https://github.com/MEDSL/2022-elections-official/blob/ad541ea94cb6697c0375a8cffe8e91b60372f50b/README.md
    # Per the Nevada Secretary of State, "*"" in the Votes column indicates low
    # turnout and is hidden to protect voter privacy."
    "votes": "string",
    # "dataverse": "string", # one of {"STATE", "PRESIDENT", "SENATE", "HOUSE", None}
    "readme_check": "boolean",
}


def to_schema(t):
    t = t[schema.keys()]
    t = t.mutate(
        date=parse_date(t.date),
        county_fips2=parse_fips5(t.county_fips),
        jurisdiction_fips2=parse_fips5(t.jurisdiction_fips),
    )
    t = t.cast(schema)
    return t


def read_raw(p):
    # have to set quote explicitly to get around
    # https://github.com/duckdb/duckdb/issues/11838
    t = ibis.read_csv(p, all_varchar=True, quote='"')
    t = to_schema(t)
    return t

In [4]:
import zipfile

DATA_DIR = Path("../data")


def unzip_csvs(zip_path: Path, dest: Path) -> Path:
    with zipfile.ZipFile(zip_path, "r") as z:
        # extract any csvs in the zip file
        csvs = [f for f in z.namelist() if f.endswith(".csv")]
        z.extractall(dest, csvs)


def get_github_csvs() -> list[Path]:
    csv_dir = DATA_DIR / "github_2022/csvs"
    if not csv_dir.exists():
        zip_dir = DATA_DIR / "github_2022"
        for zip_path in zip_dir.glob("*.zip"):
            unzip_csvs(zip_path, csv_dir)
    csvs = sorted(csv_dir.glob("*.csv"))
    assert len(csvs) == 51, csvs  # 50 states + DC
    return csvs


def read_csvs():
    paths = [
        *get_github_csvs(),
        *(DATA_DIR / "dataverse_2018_state").glob("*.csv"),
        *(DATA_DIR / "dataverse_2018_us_house").glob("*.csv"),
        *(DATA_DIR / "dataverse_2018_us_senate").glob("*.csv"),
        *(DATA_DIR / "dataverse_2020_state").glob("*.csv"),
        *(DATA_DIR / "dataverse_2020_us_house").glob("*.csv"),
        *(DATA_DIR / "dataverse_2020_us_senate").glob("*.csv"),
        *(DATA_DIR / "dataverse_2020_president").glob("*.csv"),
    ]
    tables = [read_raw(p) for p in paths]
    return ibis.union(*tables)


t = read_csvs()
t = t.distinct()
t = t.cache()
print(t.count())
t

34263636


┏━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ year  ┃ date       ┃ state_po ┃ county_name ┃ county_fips ┃ jurisdiction_name ┃ jurisdiction_fips ┃ district ┃ office                  ┃ magnitude ┃ special ┃ stage  ┃ precinct     ┃ writein ┃ candidate          ┃ party_detailed ┃ mode   ┃ votes  ┃ readme_check ┃ county_fips2 ┃ jurisdiction_fips2 ┃
┡━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ int16 │ date       │ string   │ string      │ string      │ string            │ string            │ string   │ string                  │ int64     │ boolean │ string │ string       │ boolean │ string             │ string         │ string │ string │ boolean      │ string       │ string             │
├───────┼────────────┼──────────┼─────────────┼─────────────┼───────────────────┼───────────────────┼──────────┼─────────────────────────┼───────────┼─────────┼────────┼──────────────┼─────────┼────────────────────┼────────────────┼────────┼────────┼──────────────┼──────────────┼────────────────────┤
│  2022 │ 2022-11-08 │ IN       │ LAGRANGE    │ 18087       │ LAGRANGE          │ 18087             │ NULL     │ LAGRANGE COUNTY SHERIFF │         1 │ False   │ GEN    │ Milford West │ False   │ TRACY HARKER       │ REPUBLICAN     │ TOTAL  │ 270    │ True         │ 018087       │ 018087             │
│  2022 │ 2022-11-08 │ IN       │ LAGRANGE    │ 18087       │ LAGRANGE          │ 18087             │ NULL     │ LAGRANGE COUNTY SHERIFF │         1 │ False   │ GEN    │ Van Buren    │ False   │ JASON A EISEMAN    │ DEMOCRAT       │ TOTAL  │ 62     │ True         │ 018087       │ 018087             │
│  2022 │ 2022-11-08 │ IN       │ LAKE        │ 18089       │ LAKE              │ 18089             │ NULL     │ LAKE COUNTY SHERIFF     │         1 │ False   │ GEN    │ Cal 14       │ False   │ DAVID K CRANE JR   │ REPUBLICAN     │ TOTAL  │ 45     │ True         │ 018089       │ 018089             │
│  2022 │ 2022-11-08 │ IN       │ LAKE        │ 18089       │ LAKE              │ 18089             │ NULL     │ LAKE COUNTY SHERIFF     │         1 │ False   │ GEN    │ Cct 05       │ False   │ DAVID K CRANE JR   │ REPUBLICAN     │ TOTAL  │ 361    │ True         │ 018089       │ 018089             │
│  2022 │ 2022-11-08 │ IN       │ LAKE        │ 18089       │ LAKE              │ 18089             │ NULL     │ LAKE COUNTY SHERIFF     │         1 │ False   │ GEN    │ Cl 02        │ True    │ JAMES L JOHNSON JR │ OTHER          │ TOTAL  │ 2      │ True         │ 018089       │ 018089             │
│  2022 │ 2022-11-08 │ IN       │ LAKE        │ 18089       │ LAKE              │ 18089             │ NULL     │ LAKE COUNTY SHERIFF     │         1 │ False   │ GEN    │ Cl 07        │ True    │ JAMES L JOHNSON JR │ OTHER          │ TOTAL  │ 1      │ True         │ 018089       │ 018089             │
│  2022 │ 2022-11-08 │ IN       │ LAKE        │ 18089       │ LAKE              │ 18089             │ NULL     │ LAKE COUNTY SHERIFF     │         1 │ False   │ GEN    │ Cl 09        │ False   │ DAVID K CRANE JR   │ REPUBLICAN     │ TOTAL  │ 1      │ True         │ 018089       │ 018089             │
│  2022 │ 2022-11-08 │ IN       │ LAKE        │ 18089       │ LAKE              │ 18089             │ NULL     │ LAKE COUNTY SHERIFF     │         1 │ False   │ GEN    │ Cp 05        │ False   │ DAVID K CRANE JR   │ REPUBLICAN     │ TOTAL  │ 290    │ True         │ 018089       │ 018089             │
│  2022 │ 2022-11-08 │ IN       │ LAKE        │ 18089       │ LAKE        

# Fixup Candidates

In [5]:
t.candidate.topk(100).preview(max_rows=100)

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ candidate                   ┃ CountStar(ibis_cache_6umeumdk4ja5bpjadtzkj7yrzu) ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ string                      │ int64                                            │
├─────────────────────────────┼──────────────────────────────────────────────────┤
│ NO                          │                                          2477828 │
│ YES                         │                                          2469262 │
│ UNDERVOTES                  │                                          1352968 │
│ OVERVOTES                   │                                          1162719 │
│ WRITEIN                     │                                           806142 │
│ WRITE-IN                    │                                           645339 │
│ DONALD J TRUMP              │                                           262342 │
│ JOSEPH R BIDEN              │                                           262140 │
│ JO JORGENSEN                │                                           232039 │
│ HOWIE HAWKINS               │                                           198748 │
│ OTHER WRITE-INS             │                                           169402 │
│ UNDER VOTES                 │                                           155458 │
│ OVER VOTES                  │                                           152642 │
│ AGAINST                     │                                           109500 │
│ FOR                         │                                           109344 │
│ GAVIN NEWSOM                │                                           100567 │
│ RICARDO LARA                │                                           100567 │
│ ELENI KOUNALAKIS            │                                           100567 │
│ THOMAS P DINAPOLI           │                                            98634 │
│ GLORIA LA RIVA              │                                            95141 │
│ NULL                        │                                            90531 │
│ LETITIA A JAMES             │                                            75126 │
│ BROCK PIERCE                │                                            70834 │
│ BLANK BALLOTS               │                                            70075 │
│ DON BLANKENSHIP             │                                            62784 │
│ KIRSTEN E GILLIBRAND        │                                            59715 │
│ NONE OF THESE CANDIDATES    │                                            56812 │
│ ANDREW M CUOMO              │                                            56067 │
│ BRIAN DAHLE                 │                                            55715 │
│ BLANK                       │                                            55204 │
│ ROB BONTA                   │                                            54191 │
│ NATHAN HOCHMAN              │                                            53301 │
│ ANGELA E UNDERWOOD JACOBS   │                                            53301 │
│ MALIA M COHEN               │                                            53301 │
│ LANHEE J CHEN               │                                            53301 │
│ ROBERT HOWELL               │                                            53301 │
│ KANYE WEST                  │                                            52457 │
│ ROCQUE "ROCKY" DE LA FUENTE │                                            48013 │
│ DIANNE FEINSTEIN            │                                            47266 │
│ JOHN H COX                  │                                            47266 │
│ XAVIER BECERRA              │                                            47266 │
│ GREG CONLON                 │                                            47266 │
│ KONSTANTINOS RODITIS        │                                            47266 │
│ ED HERNANDEZ  

In [6]:
cand_mapping = {
    "FOR": "YES",
    "AGAINST": "NO",
    "UNDER VOTES": "UNDERVOTES",
    "OVER VOTES": "OVERVOTES",
    "WRITE-IN": "WRITEIN",
    "NONE OF THESE CANDIDATES": "OTHER",
    "OTHER WRITE-INS": "OTHER",
    "BLANK BALLOTS": "BLANK",
    "BLANK/VOID": "BLANK",
}
t = t.mutate(candidate=t.candidate.substitute(cand_mapping))

## Fixup District

In [7]:
is_statewide = _.office.isin(["GOVERNOR", "US SENATE", "US PRESIDENT"])

t.filter(
    is_statewide,
).group_by("office", _.district.fillna("NULL").name("district")).agg(
    n=_.count(),
).order_by(
    _.office,
    _.n.asc(),
).preview(max_rows=40)

┏━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┓
┃ office       ┃ district  ┃ n       ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━┩
│ string       │ string    │ int64   │
├──────────────┼───────────┼─────────┤
│ GOVERNOR     │ 026       │       3 │
│ GOVERNOR     │ 000       │   82270 │
│ GOVERNOR     │ STATEWIDE │  972138 │
│ GOVERNOR     │ NULL      │  990862 │
│ US PRESIDENT │ 011       │     668 │
│ US PRESIDENT │ 008       │     700 │
│ US PRESIDENT │ 010       │     884 │
│ US PRESIDENT │ 007       │     968 │
│ US PRESIDENT │ 006       │    1068 │
│ US PRESIDENT │ 004       │    1144 │
│ US PRESIDENT │ 005       │    1412 │
│ US PRESIDENT │ 009       │    1568 │
│ US PRESIDENT │ 003       │    3047 │
│ US PRESIDENT │ 001       │    3325 │
│ US PRESIDENT │ 002       │    4382 │
│ US PRESIDENT │ NULL      │   54299 │
│ US PRESIDENT │ STATEWIDE │ 1909115 │
│ US SENATE    │ 011       │     501 │
│ US SENATE    │ 008       │     525 │
│ US SENATE    │ 002       │     579 │
│ US SENATE    │ 003       │     645 │
│ US SENATE    │ 010       │     663 │
│ US SENATE    │ 007       │     726 │
│ US SENATE    │ 001       │     792 │
│ US SENATE    │ 006       │     801 │
│ US SENATE    │ 004       │     858 │
│ US SENATE    │ 005       │    1059 │
│ US SENATE    │ 009       │    1176 │
│ US SENATE    │ 000       │   90599 │
│ US SENATE    │ NULL      │  889239 │
│ US SENATE    │ STATEWIDE │ 1392022 │
└──────────────┴───────────┴─────────┘

In [8]:
t = t.mutate(district=is_statewide.ifelse("STATEWIDE", _.district))
# also replace "AT-LARGE" with "STATEWIDE"
t = t.mutate(district=_.district.replace("AT-LARGE", "STATEWIDE"))
t.district.topk(20)

┏━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ district  ┃ CountStar() ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ string    │ int64       │
├───────────┼─────────────┤
│ STATEWIDE │    14536726 │
│ NULL      │     8278056 │
│ 000       │     1216860 │
│ 001       │      755403 │
│ 002       │      647345 │
│ 003       │      634513 │
│ 004       │      559351 │
│ 005       │      386455 │
│ 006       │      322842 │
│ 007       │      305851 │
│ …         │           … │
└───────────┴─────────────┘

In [9]:
# fixup districts that should be statewide but are NULL
is_dc_congress = (_.state_po == "DC") & (_.office == "US HOUSE")
is_ak_congress = (_.state_po == "AK") & (_.office == "US HOUSE")
is_statewide = is_dc_congress | is_ak_congress
t = t.mutate(district=is_statewide.ifelse("STATEWIDE", _.district))

## Fix Mode

In [10]:
t.mode.topk(30).preview(max_rows=30)

┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ mode                 ┃ CountStar() ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ string               │ int64       │
├──────────────────────┼─────────────┤
│ TOTAL                │    20220271 │
│ ELECTION DAY         │     3301064 │
│ ABSENTEE             │     2707749 │
│ PROVISIONAL          │     1562572 │
│ NOT ABSENTEE         │     1252855 │
│ EARLY                │      736020 │
│ ABSENTEE BY MAIL     │      630956 │
│ UNSPECIFIED          │      451357 │
│ ONE STOP             │      443871 │
│ EARLY VOTING         │      330005 │
│ IN-PERSON            │      261275 │
│ EARLY VOTE           │      208653 │
│ MAIL                 │      188146 │
│ WRITE-IN             │      154663 │
│ MAIL-IN CANVASS 1    │      128010 │
│ MAIL-IN CANVASS 2    │      128010 │
│ ADVANCED             │      124192 │
│ NULL                 │      123846 │
│ ADVANCED VOTING      │      117774 │
│ EARLY VOTING - IVO   │       96585 │
│ EARLY VOTING - OSS   │       96585 │
│ ELECTION DAY - OSS   │       96585 │
│ ELECTION DAY - IVO   │       96585 │
│ 2ND ABSENTEE         │       89340 │
│ ADVANCE IN PERSON    │       84295 │
│ FAILSAFE             │       78608 │
│ FAILSAFE PROVISIONAL │       75894 │
│ IN PERSON ABSENTEE   │       41447 │
│ MACHINE              │       39147 │
│ ADVANCE IN PERSON 2  │       38006 │
└──────────────────────┴─────────────┘

In [11]:
mode_mapping = {
    "EARLY VOTE": "EARLY",
    "EARLY VOTING": "EARLY",
    "ADVANCED VOTING": "EARLY",
    "ADVANCED": "EARLY",
}
t = t.mutate(mode=_.mode.substitute(mode_mapping))

In [12]:
t.filter(
    _.mode.isnull(),
).state_po.topk(10)

┏━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ state_po ┃ CountStar() ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ string   │ int64       │
├──────────┼─────────────┤
│ IN       │       87363 │
│ MT       │       18617 │
│ OH       │       17866 │
└──────────┴─────────────┘

In [13]:
def add_n_rows_per_cand(t):
    return (
        t.group_by(
            _.year,
            _.state_po,
            _.office,
            _.district,
            _.county_name,
            _.precinct,
            _.candidate,
        )
        .mutate(
            n_per_candidate=_.count(),
        )
        .order_by(
            _.n_per_candidate.desc(),
        )
    )


add_n_rows_per_cand(t).filter(_.mode.isnull())

┏━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ year  ┃ date       ┃ state_po ┃ county_name ┃ county_fips ┃ jurisdiction_name ┃ jurisdiction_fips ┃ district ┃ office         ┃ magnitude ┃ special ┃ stage  ┃ precinct       ┃ writein ┃ candidate  ┃ party_detailed ┃ mode   ┃ votes  ┃ readme_check ┃ county_fips2 ┃ jurisdiction_fips2 ┃ n_per_candidate ┃
┡━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ int16 │ date       │ string   │ string      │ string      │ string            │ string            │ string   │ string         │ int64     │ boolean │ string │ string         │ boolean │ string     │ string         │ string │ string │ boolean      │ string       │ string             │ int64           │
├───────┼────────────┼──────────┼─────────────┼─────────────┼───────────────────┼───────────────────┼──────────┼────────────────┼───────────┼─────────┼────────┼────────────────┼─────────┼────────────┼────────────────┼────────┼────────┼──────────────┼──────────────┼────────────────────┼─────────────────┤
│  2018 │ 2018-11-06 │ IN       │ JOHNSON     │ 18081       │ JOHNSON           │ 18081             │ 001      │ COUNTY COUNCIL │      NULL │ False   │ GEN    │ 0032 NEEDHAM 1 │ False   │ UNDERVOTES │ NULL           │ NULL   │ 855    │ True         │ 018081       │ 018081             │               3 │
│  2018 │ 2018-11-06 │ IN       │ JOHNSON     │ 18081       │ JOHNSON           │ 18081             │ 001      │ COUNTY COUNCIL │      NULL │ False   │ GEN    │ 0032 NEEDHAM 1 │ False   │ UNDERVOTES │ NULL           │ NULL   │ 100    │ True         │ 018081       │ 018081             │               3 │
│  2018 │ 2018-11-06 │ IN       │ JOHNSON     │ 18081       │ JOHNSON           │ 18081             │ 001      │ COUNTY COUNCIL │      NULL │ False   │ GEN    │ 0032 NEEDHAM 1 │ False   │ UNDERVOTES │ NULL           │ NULL   │ 91     │ True         │ 018081       │ 018081             │               3 │
│  2018 │ 2018-11-06 │ IN       │ JOHNSON     │ 18081       │ JOHNSON           │ 18081             │ 001      │ COUNTY COUNCIL │      NULL │ False   │ GEN    │ 0036 NEEDHAM 5 │ False   │ UNDERVOTES │ NULL           │ NULL   │ 138    │ True         │ 018081       │ 018081             │               3 │
│  2018 │ 2018-11-06 │ IN       │ JOHNSON     │ 18081       │ JOHNSON           │ 18081             │ 001      │ COUNTY COUNCIL │      NULL │ False   │ GEN    │ 0036 NEEDHAM 5 │ False   │ UNDERVOTES │ NULL           │ NULL   │ 873    │ True         │ 018081       │ 018081             │               3 │
│  2018 │ 2018-11-06 │ IN       │ JOHNSON     │ 18081       │ JOHNSON           │ 18081             │ 001      │ COUNTY COUNCIL │      NULL │ False   │ GEN    │ 0036 NEEDHAM 5 │ False   │ UNDERVOTES │ NULL           │ NULL   │ 131    │ True         │ 018081       │ 018081             │               3 │
│  2018 │ 2018-11-06 │ IN       │ JOHNSON     │ 18081       │ JOHNSON           │ 18081             │ 001      │ COUNTY COUNCIL │      NULL │ False   │ GEN    │ 0035 NEEDHAM 4 │ False   │ UNDERVOTES │ NULL           │ NULL   │ 754    │ True         │ 018081       │ 018081             │               3 │
│  2018 │ 2018-11-06 │ IN       │ JOHNSON     │ 18081       │ JOHNSON           │ 18081             │ 001      │ COUNTY COUNCIL │      NULL │ False   │ GEN    │ 0035 NEEDHAM 4 │ False   │ UNDERVOTES │ NULL           │ NULL   │ 108    │ True         │ 018081       │ 018081             │               3 │
│  2018 │ 2018-11-06 │ IN       │ J

In [14]:
# OK, that looks like all the mode=NULL rows should actually be "TOTAL",
# there is only one row per candidate per precinct. If a candidate
# had more than one row per precinct, then one of them might be TOTAL,
# and the rest would be something else. But that's not the case here.
fixable = ibis.and_(
    _.mode.isnull(),
    _.n_per_candidate == 1,
)
t = (
    add_n_rows_per_cand(t)
    .mutate(mode=fixable.ifelse("TOTAL", _.mode))
    .drop("n_per_candidate")
)
t = t.cache()
t.filter(
    _.mode.isnull(),
).state_po.topk(10)

┏━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ state_po ┃ CountStar() ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ string   │ int64       │
├──────────┼─────────────┤
│ IN       │          17 │
└──────────┴─────────────┘

## Save to file

In [15]:
# order both so its more user friendly,
# and it might be smaller due to better compression
t = t.order_by(
    _.year,
    _.state_po,
    _.office,
    _.county_name,
    _.district,
    _.precinct,
    _.candidate,
    _.mode,
)
t = t.cache()
t

┏━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ year  ┃ date       ┃ state_po ┃ county_name ┃ county_fips ┃ jurisdiction_name ┃ jurisdiction_fips ┃ district  ┃ office                        ┃ magnitude ┃ special ┃ stage  ┃ precinct               ┃ writein ┃ candidate ┃ party_detailed ┃ mode         ┃ votes  ┃ readme_check ┃ county_fips2 ┃ jurisdiction_fips2 ┃
┡━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ int16 │ date       │ string   │ string      │ string      │ string            │ string            │ string    │ string                        │ int64     │ boolean │ string │ string                 │ boolean │ string    │ string         │ string       │ string │ boolean      │ string       │ string             │
├───────┼────────────┼──────────┼─────────────┼─────────────┼───────────────────┼───────────────────┼───────────┼───────────────────────────────┼───────────┼─────────┼────────┼────────────────────────┼─────────┼───────────┼────────────────┼──────────────┼────────┼──────────────┼──────────────┼────────────────────┤
│  2018 │ 2018-11-06 │ AK       │ NULL        │ NULL        │ DISTRICT 1        │ 2001.0            │ STATEWIDE │ BALLOT MEASURE NO. 1 - 17FSH2 │         1 │ False   │ GEN    │ 01-446 AURORA          │ False   │ NO        │ NULL           │ ELECTION DAY │ 570    │ False        │ NULL         │ 02001              │
│  2018 │ 2018-11-06 │ AK       │ NULL        │ NULL        │ DISTRICT 1        │ 2001.0            │ STATEWIDE │ BALLOT MEASURE NO. 1 - 17FSH2 │         1 │ False   │ GEN    │ 01-446 AURORA          │ False   │ YES       │ NULL           │ ELECTION DAY │ 274    │ False        │ NULL         │ 02001              │
│  2018 │ 2018-11-06 │ AK       │ NULL        │ NULL        │ DISTRICT 1        │ 2001.0            │ STATEWIDE │ BALLOT MEASURE NO. 1 - 17FSH2 │         1 │ False   │ GEN    │ 01-455 FAIRBANKS NO. 1 │ False   │ NO        │ NULL           │ ELECTION DAY │ 99     │ False        │ NULL         │ 02001              │
│  2018 │ 2018-11-06 │ AK       │ NULL        │ NULL        │ DISTRICT 1        │ 2001.0            │ STATEWIDE │ BALLOT MEASURE NO. 1 - 17FSH2 │         1 │ False   │ GEN    │ 01-455 FAIRBANKS NO. 1 │ False   │ YES       │ NULL           │ ELECTION DAY │ 53     │ False        │ NULL         │ 02001              │
│  2018 │ 2018-11-06 │ AK       │ NULL        │ NULL        │ DISTRICT 1        │ 2001.0            │ STATEWIDE │ BALLOT MEASURE NO. 1 - 17FSH2 │         1 │ False   │ GEN    │ 01-465 FAIRBANKS NO. 2 │ False   │ NO        │ NULL           │ ELECTION DAY │ 167    │ False        │ NULL         │ 02001              │
│  2018 │ 2018-11-06 │ AK       │ NULL        │ NULL        │ DISTRICT 1        │ 2001.0            │ STATEWIDE │ BALLOT MEASURE NO. 1 - 17FSH2 │         1 │ False   │ GEN    │ 01-465 FAIRBANKS NO. 2 │ False   │ YES       │ NULL           │ ELECTION DAY │ 100    │ False        │ NULL         │ 02001              │
│  2018 │ 2018-11-06 │ AK       │ NULL        │ NULL        │ DISTRICT 1        │ 2001.0            │ STATEWIDE │ BALLOT MEASURE NO. 1 - 17FSH2 │         1 │ False   │ GEN    │ 01-470 FAIRBANKS NO. 3 │ False   │ NO        │ NULL           │ ELECTION DAY │ 325    │ False        │ NULL         │ 02001              │
│  2018 │ 2018-11-06 │ AK       │ NULL        │ NULL        │ DISTRICT 1        │ 2001.0            │ STATEWIDE │ BALLOT MEASURE NO. 1 - 17FSH2 │         1 │ False   │ GEN    │ 01-470 FAIRBANKS NO. 3 │ False 

In [16]:
def check_integrity(t):
    assert t.year.isin([2018, 2020, 2022]).all().execute()
    assert not (t.district == "NULL").any().execute()


check_integrity(t)

In [18]:
t.to_parquet(DATA_DIR / "cleaned.parquet")